### **MACHINE LEARNING**
 Training and testing historical data to make prediction
 
 Core logic Mainly use XGBoost( eXtreme Gradient Boosting) framework


In [14]:
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error


#Load and Prepare Data
print("Loading csv...")
df = pd.read_csv('PP_test5nasa.csv', skiprows=16)

# Create date column from YEAR, MO, DY
df = df.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'})
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.set_index('date').sort_index()

#Handle missing values (-999)
df = df.replace(-999, np.nan)
df = df.dropna()  # Drop rows with NaN (last few days are missing)

#Convert wind speed from m/s to km/h (consistent with previous datasets)
df['wind_speed_10m_kmh'] = df['WS10M'] * 3.6


# Additional features: humidity, pressure, wind direction
df['wind_dir_rad'] = np.deg2rad(df['WD10M'])
df['wind_dir_sin'] = np.sin(df['wind_dir_rad'])
df['wind_dir_cos'] = np.cos(df['wind_dir_rad'])

print(f"Loaded {len(df)} valid days up to {df.index[-1].date()}")

# #load data and clean data 
# def loaddata(filename):
#     print(f'loading data from {filename}')
#     # Try reading with skiprows=17, if columns missing, try without skipping
#     try:
#         df = pd.read_csv(filename, skiprows=16)
#         df.columns = [col.upper() for col in df.columns]  # Standardize column names to uppercase
#         if not all(col in df.columns for col in ['YEAR', 'MO', 'DY']):
#             # Try reading without skipping rows
#             df = pd.read_csv(filename)
#             df.columns = [col.upper() for col in df.columns]
#     except Exception as e:
#         print("Error reading CSV:", e)
#         df = pd.read_csv(filename)
#         df.columns = [col.upper() for col in df.columns]

#     # Check if year, month, day columns exist
#     if not all(col in df.columns for col in ['year', 'month', 'day']):
#         raise KeyError("CSV file must contain 'year', 'month', 'day' columns for date construction.")

#     # Create date column from year, month, day
#     df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
#     df = df.set_index('date').sort_index()

#     # Handle missing values (-999)
#     df = df.replace(-999, np.nan)
#     df = df.dropna()  # Drop rows with NaN (last few day are missing)
#     return df


targets = [
        'T2M_MAX', #max temp 2m in °C
        'T2M_MIN', #min temp 2m in °C
        'T2M', #mean temp
        'RH2M',  #Relative Humidity at 2 Meters (%)
        'WS10M',    #Wind Speed at 10 Meters (m/s)
        'WD10M',   #Wind Direction at 10 Meters (Degrees)
        'PRECTOTCORR' #Precipitation (mm/day)
        ]

def engineer_features(df):
    df = df.copy()
    
    # Lags: Past 1-7 days for all targets
    for col in targets:
        for lag in range(1, 8):
            df[f'{col}_lag{lag}'] = df[col].shift(lag)
    
    # Rolling stats
    df['temp_max_roll_mean_3'] = df['T2M_MAX'].rolling(3).mean()
    df['temp_max_roll_mean_7'] = df['T2M_MAX'].rolling(7).mean()
    df['precip_roll_sum_7'] = df['PRECTOTCORR'].rolling(7).sum()
    df['humidity_roll_mean_7'] = df['RH2M'].rolling(7).mean()  # Add humidity trend
    df['pressure_roll_mean_7'] = df['PS'].rolling(7).mean()    # Pressure trend
    
    # Date features (seasonality)
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    
    # Drop NaN from shifts/rolling
    return df.dropna()

data = engineer_features(df)
    

Loading csv...
Loaded 327 valid days up to 2025-11-23


In [15]:

#Train with XGbosst 

feature_cols = [col for col in data.columns 
                if col not in targets + ['WD10M', 'wind_dir_rad', 'WS10M']]  # Exclude raw wind dir, speed

models = {}
print("Training XGBoost models...")
for target in targets:
    print(f"  → Training {target}...")
    model = xgb.XGBRegressor(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        tree_method='hist',  
    )
    model.fit(data[feature_cols], data[target])
    models[target] = model
    
    # Quick accuracy check on full data
    mae = mean_absolute_error(data[target], model.predict(data[feature_cols]))
    print(f"     MAE: {mae:.2f}")

last_row = data.iloc[-1]  # Last valid day for forecasting start
print("Training complete!")

Training XGBoost models...
  → Training T2M_MAX...
     MAE: 0.00
  → Training T2M_MIN...
     MAE: 0.00
  → Training T2M...
     MAE: 0.00
  → Training RH2M...
     MAE: 0.01
  → Training WS10M...
     MAE: 0.00
  → Training WD10M...
     MAE: 0.02
  → Training PRECTOTCORR...
     MAE: 0.01
Training complete!


In [16]:
#Direction
def degrees_to_direction(deg):
    #Convert degrees → N, NE, E, SE, S, SW, W, NW
    
    deg = deg % 360 #just making sure degree is degree
    if deg < 22.5 or deg >= 337.5:   return "N"
    elif deg < 67.5:                 return "NE"
    elif deg < 112.5:                return "E"
    elif deg < 157.5:                return "SE"
    elif deg < 202.5:                return "S"
    elif deg < 247.5:                return "SW"
    elif deg < 292.5:                return "W"
    else:                            return "NW"

In [18]:
#Forecast Next 7 Days (Recursive)
def forecast_next_7_days(start_row, models, feature_cols, days=7):
    current = start_row.copy()
    today = datetime.now().date()
    dates = [today + timedelta(days=i) for i in range(days + 1)]  # Today + 7 days
    forecasts = []

    for i in range(days + 1):
        X = current[feature_cols].values.reshape(1, -1)
        pred = {}
        for target in targets:
            pred[target] = models[target].predict(X)[0]
        pred['PRECTOTCORR'] = max(0, pred['PRECTOTCORR'])
        pred['WD10M'] = current['WD10M']
        forecasts.append(pred)

        # Update for next day
        new_row = current.copy()
        for t in targets:
            new_row[t] = pred[t]
        
        next_date = current.name + timedelta(days=1)
        new_row.name = next_date
        new_row['month'] = next_date.month
        new_row['day'] = next_date.day
        new_row['dayofweek'] = next_date.dayofweek
        new_row['is_weekend'] = 1 if next_date.weekday() >= 5 else 0

        # Update lags (shift forward)
        for t in targets:
            for lag in range(7, 1, -1):
                new_row[f'{t}_lag{lag}'] = current[f'{t}_lag{lag-1}']
            new_row[f'{t}_lag1'] = pred[t]

        # Update rolling features (approximate with recent lags)
        new_row['temp_max_roll_mean_3'] = np.mean([
            pred['T2M_MAX'], current['T2M_MAX_lag1'], current['T2M_MAX_lag2']
        ])
        new_row['temp_max_roll_mean_7'] = np.mean([
            pred['T2M_MAX']] + [current[f'T2M_MAX_lag{i}'] for i in range(1, 7)]
        )
        new_row['precip_roll_sum_7'] = sum([
            pred['PRECTOTCORR']] + [current[f'PRECTOTCORR_lag{i}'] for i in range(1, 7)]
        )
        new_row['humidity_roll_mean_7'] = np.mean([
            current['RH2M']] + [current[f'RH2M_lag{i}'] for i in range(1, 7)]  # Approximate
        )
        # No PS_lag features available, so just use current value for rolling mean
        new_row['pressure_roll_mean_7'] = current['PS']

        # Update wind sin/cos
        rad = np.deg2rad(pred['WD10M'])
        new_row['wind_dir_sin'] = np.sin(rad)
        new_row['wind_dir_cos'] = np.cos(rad)

        current = new_row

    forecast_df = pd.DataFrame(forecasts, index=dates).round(1)

    # Add weather description
    desc = []
    for _, row in forecast_df.iterrows():
        precip = row['PRECTOTCORR']
        temp_max = row['T2M_MAX']
        if precip < 0.5:
            desc.append("Sunny" if temp_max >= 33 else "Mostly Sunny" if temp_max >= 30 else "Partly Cloudy")
        elif precip < 3:
            desc.append("Light Rain")
        elif precip < 10:
            desc.append("Rainy")
        else:
            desc.append("Heavy Rain")
    forecast_df['Weather'] = desc

    return forecast_df

#Save forecast to CSV
#forecast.to_csv('phnom_penh_forecast_nov2025.csv')
#print("\nForecast saved to 'phnom_penh_forecast_nov2025.csv'")

In [20]:
# Generate forecast
forecast = forecast_next_7_days(last_row, models, feature_cols)
forecast['Wind_Direction'] = forecast['WD10M'].apply(degrees_to_direction)

#Display Results
print("\n7-DAY WEATHER FORECAST FOR PHNOM PENH")
print("=" * 100)
print(f"{'Date':<12} {'Max':>6} {'Min':>6} {'Mean':>7} {'Rain':>7} {'Wind':>10} {'Dir':>5} {'Weather':<15}")
print("-"*85)

for date, row in forecast.iterrows():
    print(f"{date.strftime('%a %b %d'):<12} "
          f"{row['T2M_MAX']:>5.1f}° "
          f"{row['T2M_MIN']:>5.1f}° "
          f"{row['T2M']:>6.1f}° "
          f"{row['PRECTOTCORR']:>5.1f}mm "
          f"{row['WS10M'] * 3.6:>6.1f} km/h "
          f"{row['Wind_Direction']:>5}   "
          f"{row['Weather']:<15}")

print("="*85)
print("Model: XGBoost | Data: NASA POWER | Accuracy: MAE --")
print("="*85)



7-DAY WEATHER FORECAST FOR PHNOM PENH
Date            Max    Min    Mean    Rain       Wind   Dir Weather        
-------------------------------------------------------------------------------------
Sun Nov 30    29.0°  22.1°   25.2°   4.8mm   18.7 km/h     N   Rainy          
Mon Dec 01    27.7°  22.0°   24.6°   4.9mm   19.1 km/h     N   Rainy          
Tue Dec 02    27.6°  21.6°   25.0°   3.6mm   19.1 km/h     N   Rainy          
Wed Dec 03    27.6°  21.0°   24.8°   2.8mm   19.1 km/h     N   Light Rain     
Thu Dec 04    26.8°  21.0°   24.8°   2.3mm   19.1 km/h     N   Light Rain     
Fri Dec 05    27.2°  20.9°   24.9°   2.3mm   19.1 km/h     N   Light Rain     
Sat Dec 06    27.3°  21.1°   24.5°   2.4mm   19.1 km/h     N   Light Rain     
Sun Dec 07    27.6°  21.4°   24.6°   2.8mm   19.1 km/h     N   Light Rain     
Model: XGBoost | Data: NASA POWER | Accuracy: MAE --
